In [ ]:
import gradio as gr
import random
from PIL import Image
from torchvision import transforms

import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms, models

In [ ]:
# Inference Code
# class MultiLabelDenseNet(nn.Module):
#     def __init__(self, num_classes1=3, num_classes2=3):
#         super(MultiLabelDenseNet, self).__init__()
#         self.base_model = models.densenet121(pretrained=True)
#         num_features = self.base_model.classifier.in_features
#         self.base_model.classifier = nn.Identity()
#         self.fc1 = nn.Linear(num_features, num_classes1)
#         self.fc2 = nn.Linear(num_features, num_classes2)

#     def forward(self, x):
#         features = self.base_model(x)
#         return self.fc1(features), self.fc2(features)
    
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# # Load the model
# model = MultiLabelDenseNet()
# model.load_state_dict(torch.load('./trained_model/densenet_epoch_2.pth'))
# model.eval()

# Define the Model using Vision Transformer
class MultiLabelViT(nn.Module):
    def __init__(self, num_classes1=3, num_classes2=3):
        super(MultiLabelViT, self).__init__()
        # Load a pretrained ViT model (ViT-B/16)
        self.base_model = models.vit_b_16(pretrained=True)
        # Obtain the number of features from the original classification head,
        # which is stored in a Sequential container (index 0)
        num_features = self.base_model.heads[0].in_features
        # Remove the original classification head by replacing it with Identity
        self.base_model.heads = nn.Identity()
        # Add three separate fully connected heads for multi-label outputs
        self.fc1 = nn.Linear(num_features, num_classes1)
        self.fc2 = nn.Linear(num_features, num_classes2)

    def forward(self, x):
        # Forward pass through the ViT model to obtain features.
        features = self.base_model(x)
        # If the returned features include a token sequence, extract the class token (first token)
        if features.dim() == 3:
            features = features[:, 0]
        # Pass the features through the three separate heads
        return self.fc1(features), self.fc2(features)

# # Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MultiLabelViT().to(device)
model.load_state_dict(torch.load('./trained_model/vit_epoch_v2_1.pth'))
model.eval()

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


def inference(input_tensor):

    with torch.no_grad():
        outputs1, outputs2 = model(input_tensor)
        _, predicted_grade = outputs1.max(1)
        _, predicted_stage = outputs2.max(1)

    return predicted_grade.item(), predicted_stage.item()

In [ ]:
inv_grade_map = {2: "A", 1: "B", 0: "C"}
inv_stage_map = {2: "8_cell", 1: "morula", 0: "blastocyte"}

def predict_embryo(image: Image.Image):
    input_tensor = preprocess(image).unsqueeze(0).to(device)
    
    grade_num, stage_num = inference(input_tensor)
    
    grade_letter = inv_grade_map.get(grade_num, "Unknown")
    stage = inv_stage_map.get(stage_num, "Unknown")
    return stage, grade_letter

tab1 = gr.Textbox(label="Developmental Stage")
tab2 = gr.Textbox(label="Embryo Grade")

app = gr.Interface(
    fn=predict_embryo,
    inputs=gr.Image(type="pil", label="Upload Embryo Microscopic Image"),
    outputs=[tab1, tab2],
    title="Embryo Quality Detection",
    description=(
        "Upload a microscopic image of an embryo to predict its quality (grade) "
        "and developmental stage."
    )
)

if __name__ == "__main__":
    app.launch()